In [15]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import faculty.datasets as datasets
from pandas.api.types import CategoricalDtype
import matplotlib.ticker as mtick

from pandas import Series, DataFrame
import os
from functools import reduce

### Import the market_value dataset

In [16]:
#import market value dataset and all football 18 one
market_value = pd.read_csv('/project/market_value.csv')
all_football_18 = pd.read_csv('/project/all_football_18.csv')

In [17]:
market_value.head()

,#,country,confederation,squad_size 2,avg_age 3,market_value
0,1,England,UEFA,25th,25.4,€ 1.23 billion
1,2,France,UEFA,24,27.3,€ 978.80 million
2,3,Brazil,CONMEBOL,23,26.5,€ 870.50 million
3,4th,Portugal,UEFA,23,27.3,€ 775.00 million
4,5,Argentina,CONMEBOL,36,25.9,€ 747.50 million


### Clean the market_value dataset

In [18]:
# Dropping unecessary columns
market_value = market_value.drop(columns=["#"])

# In order to be easier to use, we sorted the table alphabetically based on 
# the country column.
market_value = market_value.sort_values("country")
market_value = market_value.reset_index(drop=True)
market_value.head()

,country,confederation,squad_size 2,avg_age 3,market_value
0,Albania,UEFA,27,26.5,€ 77.35 million
1,Algerien,CAF,24,28.7,"32,80 Mio. €"
2,Angola,CAF,23,24.7,"21,30 Mio. €"
3,Argentina,CONMEBOL,36,25.9,€ 747.50 million
4,Armenien,UEFA,28,27.9,"41,43 Mio. €"


In [19]:
#displaying the all_football_18 dataset so that we can get a list of the countries
# but dropping the unnamed: 0 column
all_football_18 = all_football_18.drop(columns=["Unnamed: 0"])
all_football_18.head()

,country_name,Final_positional_ranking,goal,shotsPerGame,yellow_card,red_card,possession_percentage,passSuccess_percentage,shotOnTargetPG,dribbleWonPG,...,keyPassShort,left_side,middle_side,right_side,own_third,middle_third,opposition_third,shot_left_side,shot_middle_side,shot_right_side
0,Argentina,16,6,13.8,11,0,61.1,85.0,4.5,13.5,...,8.8,38%,27%,35%,19%,49%,32%,27%,49%,24%
1,Australia,25,2,11.0,7,0,50.9,83.7,2.3,11.0,...,6.3,35%,20%,45%,27%,47%,25%,9%,79%,12%
2,Belgium,3,16,15.1,11,0,52.1,86.3,5.7,13.3,...,10.6,34%,26%,40%,27%,47%,26%,26%,56%,18%
3,Brazil,5,8,20.8,7,0,56.8,87.9,8.0,14.8,...,13.0,43%,29%,29%,25%,44%,31%,28%,60%,13%
4,Colombia,10,6,9.8,9,1,50.2,79.9,3.3,7.8,...,5.0,33%,22%,45%,29%,48%,23%,7%,66%,27%


In [20]:
# Extracting all the 32 countries from the WorldCup and make a list with them in order to use them.
list_of_countries = list(all_football_18["country_name"])
list_of_countries


['Argentina',
 'Australia',
 'Belgium',
 'Brazil',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Denmark',
 'Egypt',
 'England',
 'France',
 'Germany',
 'Iceland',
 'Iran',
 'Japan',
 'Mexico',
 'Morocco',
 'Nigeria',
 'Panama',
 'Peru',
 'Poland',
 'Portugal',
 'Russia',
 'Saudi Arabia',
 'Senegal',
 'Serbia',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerland',
 'Tunisia',
 'Uruguay']

In [21]:
market_value_all = market_value[market_value['country'].isin(list_of_countries)]

market_value_all = market_value_all.reset_index(drop=True)
market_value_all.head()


,country,confederation,squad_size 2,avg_age 3,market_value
0,Argentina,CONMEBOL,36,25.9,€ 747.50 million
1,Belgium,UEFA,23,27.8,€ 419.00 million
2,Brazil,CONMEBOL,23,26.5,€ 870.50 million
3,Colombia,CONMEBOL,26th,28.5,€ 282.95 million
4,Costa Rica,CONCACAF,23,26.4,"24,25 Mio. €"


After running this, we can see that we are missing 6 countries. We needed a total of 32 but we only got 26. 

To find out which countries we are missing, we merged both tables with a "right"
join so that we can see which countries from the all_football_18 dataset didn't
have a match. **The ones missing should have a NaN value.**

In [22]:
missing_countries = pd.merge(market_value_all, all_football_18, 
  left_on='country', 
  right_on='country_name',
    how="right")

# by running below command, we want to display the rows which have NaN values to 
# see which countries they are.
missing_countries = missing_countries[missing_countries.isna().any(axis=1)]
missing_countries

,country,confederation,squad_size 2,avg_age 3,market_value,country_name,Final_positional_ranking,goal,shotsPerGame,yellow_card,...,keyPassShort,left_side,middle_side,right_side,own_third,middle_third,opposition_third,shot_left_side,shot_middle_side,shot_right_side
1,NaN,NaN,NaN,NaN,NaN,Australia,25,2,11.0,7,...,6.3,35%,20%,45%,27%,47%,25%,9%,79%,12%
8,NaN,NaN,NaN,NaN,NaN,Egypt,31,2,9.3,5,...,4.7,40%,25%,36%,33%,43%,24%,25%,61%,14%
12,NaN,NaN,NaN,NaN,NaN,Iceland,27,2,11.0,3,...,6.0,34%,31%,35%,29%,45%,25%,21%,58%,21%
18,NaN,NaN,NaN,NaN,NaN,Panama,32,2,8.0,11,...,5.3,44%,23%,33%,26%,54%,20%,24%,60%,16%
23,NaN,NaN,NaN,NaN,NaN,Saudi Arabia,29,2,12.3,1,...,7.0,35%,23%,42%,25%,51%,25%,22%,51%,27%
30,NaN,NaN,NaN,NaN,NaN,Tunisia,28,5,12.0,4,...,7.7,36%,22%,42%,29%,47%,23%,17%,61%,22%


After seeing the output of the merge, the countries missing are Australia, Egypt, 
Iceland, Panama, Saudi-Arabia, Tunisia.
Most probably they are written differently

In [23]:
# Below we are trying to find the names of these countries to see how they are written
missing_countries_name = list(market_value['country'])
[t.split() for t in missing_countries_name if t.startswith(('Austra','Saudi','Tune','Pan'))]
[t.split() for t in missing_countries_name if t.endswith(('ypten','land'))]


[['Australien'], ['Saudi-Arabien'], ['Tunesien']]

[['England'],
 ['Finnland'],
 ['Ireland'],
 ['Island'],
 ['Neuseeland'],
 ['Nordirland'],
 ['Poland'],
 ['Scotland'],
 ['Switzerland'],
 ['Ägypten']]

In [24]:
# Now that we found the names, we want to update them to match the actual names we have
market_value['country'] = market_value['country'].replace({'Australien':'Australia',
                                                          'Ägypten':'Egypt',
                                                          'Island':'Iceland',
                                                          'Saudi-Arabien':'Saudi Arabia',
                                                          'Tunesien':'Tunisia'})

# and we will re run the command to go through the list of countries from the football
# dataset and only match the ones present in both
market_value_all = market_value[market_value['country'].isin(list_of_countries)]

market_value_all = market_value_all.reset_index(drop=True)
market_value_all.head()




,country,confederation,squad_size 2,avg_age 3,market_value
0,Argentina,CONMEBOL,36,25.9,€ 747.50 million
1,Australia,AFC,23,25.6,"32,85 Mio. €"
2,Belgium,UEFA,23,27.8,€ 419.00 million
3,Brazil,CONMEBOL,23,26.5,€ 870.50 million
4,Colombia,CONMEBOL,26th,28.5,€ 282.95 million


After running this, we can see that we are now only missing 1 country as we needed a total of 32 but we only got 31. **Panama is the missing one.** After some research, we found data for Panama so we will add it.


In [25]:
#rename the columns by adding "in_euros" part, so that we can remove the € sign from the rows
market_value_all = market_value_all.rename(columns={"market_value":"market_value_in_euros",
                                                   "squad_size 2":"squad_size","avg_age 3":"avg_age"},)

# Cleaning the dataset by replacing some values:
# 1) replacing € with nothing (so basically removing it)
# 2) replacing , with . to keep the same format for the amounts
# 3) replacing "Mio." with "million" to keep the same format 
market_value_all.market_value_in_euros = market_value_all.market_value_in_euros.str.replace('€','')
market_value_all.market_value_in_euros = market_value_all.market_value_in_euros.str.replace(",",".")
market_value_all.market_value_in_euros = market_value_all.market_value_in_euros.str.replace("Mio.","million")


# We used str.extract to pass a regex pattern to extract just the numeric parts.
# the expand=False is added so that it doesn't add a new column.
market_value_all['squad_size'] = market_value_all['squad_size'].str.extract('(\d+)', expand=False)
market_value_all.head()




<ipython-input-25-3db408d24967>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  market_value_all.market_value_in_euros = market_value_all.market_value_in_euros.str.replace("Mio.","million")


,country,confederation,squad_size,avg_age,market_value_in_euros
0,Argentina,CONMEBOL,36,25.9,747.50 million
1,Australia,AFC,23,25.6,32.85 million
2,Belgium,UEFA,23,27.8,419.00 million
3,Brazil,CONMEBOL,23,26.5,870.50 million
4,Colombia,CONMEBOL,26,28.5,282.95 million


In [26]:
#Data sources: 
#https://www.businessinsider.com/how-much-each-team-at-the-world-cup-is-worth-2018-6?r=US&IR=T#32-panama-1
#https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads#Panama

new_row = {'country':'Panama', 'confederation':"CONCACAF", 'squad_size':23, 'avg_age':28.4, 'market_value_in_euros': "10.17 million"}
#append row to the dataframe
market_value_all = market_value_all.append(new_row, ignore_index=True)
market_value_all.head()



,country,confederation,squad_size,avg_age,market_value_in_euros
0,Argentina,CONMEBOL,36,25.9,747.50 million
1,Australia,AFC,23,25.6,32.85 million
2,Belgium,UEFA,23,27.8,419.00 million
3,Brazil,CONMEBOL,23,26.5,870.50 million
4,Colombia,CONMEBOL,26,28.5,282.95 million


### Feature engineering
We are going to make a new variable by dividing the market value in euros with the squad size. This will allow us to find the average market value of players in euros for each team. This will be used later for a comparison to see if better paid players perform better. 

In [27]:
# making a new column which only contains the digits of the column
market_value_all['market_value_digits'] = market_value_all['market_value_in_euros'].str.extract('(\d+)', expand=False)

# transforming both columns that we will divide to numeric
market_value_all["market_value_digits"] = pd.to_numeric(market_value_all['market_value_digits'])
market_value_all["squad_size"] = pd.to_numeric(market_value_all['squad_size'])

# we replace the value for England because it's billion and not million
market_value_all['market_value_digits'] = market_value_all['market_value_digits'].replace({1.0:1000})
market_value_all.info()

market_value_all['avg_market_value_of_players_in_mill_euros'] = market_value_all['market_value_digits']/market_value_all['squad_size']
market_value_all['avg_market_value_of_players_in_mill_euros'] = market_value_all['avg_market_value_of_players_in_mill_euros'].round(2) 
market_value_all.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                32 non-null     object 
 1   confederation          32 non-null     object 
 2   squad_size             32 non-null     int64  
 3   avg_age                32 non-null     float64
 4   market_value_in_euros  32 non-null     object 
 5   market_value_digits    32 non-null     int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.6+ KB


,country,confederation,squad_size,avg_age,market_value_in_euros,market_value_digits,avg_market_value_of_players_in_mill_euros
0,Argentina,CONMEBOL,36,25.9,747.50 million,747,20.75
1,Australia,AFC,23,25.6,32.85 million,32,1.39
2,Belgium,UEFA,23,27.8,419.00 million,419,18.22
3,Brazil,CONMEBOL,23,26.5,870.50 million,870,37.83
4,Colombia,CONMEBOL,26,28.5,282.95 million,282,10.85


### Exporting clean dataset market_value_all

In [28]:
market_value_all.to_csv("market_value_all.csv")